In [ ]:
#PACOTES NECESSÁRIOS

In [0]:
# Importa bibliotecas necessárias
import pandas as pd
import numpy as np
import graphviz
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

### Importa as bibliotecas dos métodos

In [0]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import  model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network  import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

### Configurar limite máximo de rows e columns

In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Leitura do arquivo e tratamento de campos nulos

In [7]:
# LEITURA DO ARQUIVO
print('Lendo o arquivo...')
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Datasets/basefull.xlsx')

def LimparVazio(df):
  for column in df.columns:
        print('Substituindo null por vazio no campo:', column)
        df[column].fillna(value='VAZIO', inplace=True)

df.to_csv('basefull.csv',sep=',')
LimparVazio(df)

Lendo o arquivo...
Substituindo null por vazio no campo: cRequisicao
Substituindo null por vazio no campo: cItemRC
Substituindo null por vazio no campo: cCentro
Substituindo null por vazio no campo: cMaterial
Substituindo null por vazio no campo: tDescricao
Substituindo null por vazio no campo: cDep
Substituindo null por vazio no campo: cGrupoMercadoria
Substituindo null por vazio no campo: tFornecedor
Substituindo null por vazio no campo: cFornFixo
Substituindo null por vazio no campo: tFornFixo
Substituindo null por vazio no campo: cFornPretendido
Substituindo null por vazio no campo: tFornPretendido
Substituindo null por vazio no campo: cGrpComprador
Substituindo null por vazio no campo: cClassCont
Substituindo null por vazio no campo: cCtgItem
Substituindo null por vazio no campo: cCtgMat
Substituindo null por vazio no campo: cConcluido
Substituindo null por vazio no campo: cLib
Substituindo null por vazio no campo: cStProc
Substituindo null por vazio no campo: tAcompanhamento
Subs

### Gerar profile report depois do drop

In [0]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_file(outputfile="Requisicao data profiling - Antes.html")

### Get some basic ideas about the data

In [0]:
# # get some basic ideas about the data
# def summerize_data(df):
#     for column in df.columns:
#         print(column)
#         if df.dtypes[column] == np.object: # Categorical data
#             print (df[column].value_counts())
#         else:
#             print (df[column].describe() )
            
#         print ('\n')
    
# summerize_data(df)

### Dropa as colunas sem importância para rodar o modelo

In [10]:
def DropColumns(df, col_drop):
  for row in col_drop:
      df.drop(columns=row, inplace=True)      

# col_drop = ['cCentro','cGrupoMercadoria','cCtgItem','cClassCont','cTipoRC','cGrpComprador']
col_drop = ['cRequisicao', 'cItemRC', 'cMaterial', 'tDescricao', 'cDep',
       'tFornecedor', 'cFornFixo', 'tFornFixo',
       'cFornPretendido', 'tFornPretendido',
       'cConcluido', 'cLib', 'cStProc',
       'tAcompanhamento', 'dRemessa', 'dSolicitacao', 'cPedido', 'cItemPC',
       'nQuantidade', 'nQuantidadePC', 'cUM', 'nPreco', 'nValorTotal', 'nPor',
       'tRequisitante', 'tLoginUsuario', 'cReserva', 'cItemRM',
       'cCatalogo', 'cGrpMercadoria', 'cCtgMat', 'cCtgItem']
DropColumns(df,col_drop)

print(df.columns)

#df['cMaterial'] = (df['cMaterial']).astype(object)

Index(['cCentro', 'cGrupoMercadoria', 'cGrpComprador', 'cClassCont', 'cTipoRC',
       'cGrupoCompradorPC'],
      dtype='object')


In [11]:
colunas = ['cCentro', 'cGrupoMercadoria', 'cGrpComprador', 'cClassCont', 'cTipoRC']
target = 'cGrupoCompradorPC'

#print(df[colunas],"\n")
names = np.sort(df[target].unique())
print(names)
print(df)

['DAA' 'DAB' 'DAD' 'DAE' 'DAF' 'DAG' 'DAH' 'DAJ' 'DAK' 'DAL' 'DAM' 'DAN'
 'DAO' 'DAP' 'DAQ' 'DAR' 'DAS' 'DAU' 'DAV' 'DAW' 'DAX' 'DAY' 'DAZ' 'DAÍ'
 'DB1' 'DBA' 'DBB' 'DBC' 'DBD' 'DBE' 'DBF' 'DBG' 'DBH' 'DBJ' 'DBK' 'DBO'
 'DBP' 'DBQ' 'DBR' 'DBS' 'DBT' 'DBU' 'DBW' 'DBX' 'DBY' 'DBZ' 'DCA' 'DCB'
 'DCC' 'DCD' 'DCE' 'DCF' 'DCG' 'DCI' 'DCJ' 'DCL' 'DCM' 'DCO' 'DCP' 'DCQ'
 'DCS' 'DCT' 'DCU' 'DCV' 'DCW' 'DCX' 'DCZ' 'DDA' 'DDE' 'DDX' 'DEA' 'DEK'
 'DEM' 'DEN' 'DEP' 'DEQ' 'DET' 'DEX' 'DEY' 'DGA' 'DGE']
       cCentro cGrupoMercadoria  ... cTipoRC cGrupoCompradorPC
0         4264         MT052001  ...      ZB               DBT
1         4264         MT056001  ...      ZB               DBT
2         4264         MT052001  ...      ZB               DBT
3         4264         MT052001  ...      ZB               DBT
4         4264         MT048001  ...      ZB               DBT
...        ...              ...  ...     ...               ...
316765    4064         MT024006  ...      NB               DAF
31

In [0]:
# # get some basic ideas about the data
# def summerize_data(df):
#     for column in df.columns:
#         if df.dtypes[column] == np.object: # Categorical data
#             df[column] = pd.Categorical(df[column]).codes            
#             # print (df[column])
    
# summerize_data(df)

### Gerar profile report depois do drop

In [0]:
# profile = pandas_profiling.ProfileReport(df)
# profile.to_file(outputfile="Requisicao data profiling - Depois.html")

In [0]:
y = df[target]
X = df[colunas]

### Transformando variáveis categóricas em numéricas

#### Column tranformation

In [0]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import make_column_transformer

# column_trans = make_column_transformer(
#     (OneHotEncoder(), ['cCentro', 'cGrupoMercadoria', 'cGrpComprador', 'cClassCont', 'cTipoRC'])
#     , remainder = 'passthrough')

# column_trans.fit_transform(X)

#### One Hot Encoder

In [0]:
# from sklearn.preprocessing import OneHotEncoder

# # Create the encoder.
# encoder = OneHotEncoder(handle_unknown="ignore")
# encoder.fit(X)
# X = encoder

#### Get_Dummies

In [17]:
#from sklearn import preprocessing
y=df[target]
X = pd.get_dummies(df[colunas],sparse=True)
col_list = df.columns.tolist()
#print(X)
print(X.shape)

(316770, 758)


#### Seleciona as melhores features 

In [0]:
# from sklearn.feature_selection import SelectFromModel
# from sklearn.svm import LinearSVC
# from sklearn.linear_model import LassoCV

# #clf = LinearSVC(C=1, penalty='l1', dual=False).fit(X, df[target])
# #clf = DecisionTreeClassifier(criterion='entropy', splitter='random',  random_state = 50,max_depth=6, min_samples_leaf=5).fit(X, df[target])
# clf = LogisticRegression().fit(X, df[target])
# sfm = SelectFromModel (clf, prefit=True)
# X = sfm.transform(X)

# X.shape

#### Decomposição do PCA

In [0]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=.95, svd_solver='full')
# pca.fit(X)
# print(X.shape)

### Divide a base de treino e teste

In [20]:
# Holdout -> dividindo a base em treinamento (70%) e teste (30%), estratificada
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.30, random_state=7)

print('Treino: ', X_train.shape)
print('Teste: ', X_test.shape)

Treino:  (221739, 758)
Teste:  (95031, 758)


### Configura diversos métodos para validação

In [0]:
dt =  DecisionTreeClassifier(criterion='entropy', splitter='random',  random_state = 50,max_depth=6, min_samples_leaf=5)
dt_gini = DecisionTreeClassifier(criterion = "gini", splitter='random', random_state = 50,max_depth=6, min_samples_leaf=5)
#dt_folds = DecisionTreeClassifier(criterion='entropy', splitter='random',  random_state = 50,max_depth=6, min_samples_leaf=5 )
rf = RandomForestClassifier(n_estimators=500, random_state=100, max_depth = 5) # random forest
lr = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=3)
mlp = MLPClassifier(solver='sgd', early_stopping=True, hidden_layer_sizes=(100), activation='logistic', batch_size=100, max_iter=10000, learning_rate_init=0.1, momentum=0.2, tol=1e-10)
nb = GaussianNB(var_smoothing=1e-09)
nbm = MultinomialNB
bg = BaggingClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=100, bootstrap=True)
boost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=100, random_state=0, learning_rate = 0.1)
cb = VotingClassifier(estimators=[('LogisticRegresson', lr), ('MLP', mlp)], voting='soft')
svc=SVC(probability=True, kernel='linear')

#SVM
parameters = [
  {'C': [0.1, 0.5, 1, 10, 100, 500, 1000], 'kernel': ['poly']}, 
  {'C': [0.1, 0.5, 1, 10, 100, 500, 1000], 'gamma': [0.1, 0.001, 0.0001, 0.00001], 'kernel': ['rbf']},
]
svm = SVC(gamma='scale')
svm = GridSearchCV(svm, parameters, scoring = 'accuracy', cv=10, iid=False)

titles = ['LogisticRegresson']
methods = [lr]
# titles = ['LogisticRegresson', 'MLP', 'LogReg+MLP', 'DecisionTree']
# methods = [lr, mlp, cb, dt]
#titles = ['LogisticRegresson', 'DecisionTree', 'DecisionTree Gini', 'NaiveBayes', 'MLP', 'RF', 'LogReg+MLP', 'Bagging','AdaBossting']
#methods = [lr, dt, dt_gini, nb, mlp, rf, cb, bg, boost]

### Pipeline

In [0]:
# from sklearn.pipeline import make_pipeline
# from sklearn.model_selection import cross_val_score
# import pickle

# # pipe = make_pipeline(column_trans, dt)

# # X = column_trans.fit_transform(X)
# # cross_val_score(dt, X, y, cv=5, scoring='accuracy').mean()

# # X_train = column_trans.fit_transform(X_train)
# # cross_val_score(dt, X_train, y_train, cv=5, scoring='accuracy').mean()

# pipe.fit(X_train, y_train)


# # column_trans.fit_transform(X_test)
# # y_pred = pipe.predict(X_test)

# # print("Classification accuracy {} = {}"
# #           .format('pipe', pipe.score(X_test, y_test)))

# # pickle.dump(pipe, open("pipe" + ".pkl","wb"))
# # column_trans.fit_transform(X_train)

### Testa o treino na base de teste </p>
Compara a predição em relação ao correto

In [0]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

def exporta_graf(name, minha_arvore):
  dot_data = StringIO()
  export_graphviz(minha_arvore, out_file=dot_data,
                  filled=True, rounded=True,
                  special_characters=True,
                  feature_names = X.columns, class_names=names)
  graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
  graph.write_png(name + '.png')
  #Image(graph.create_png())

In [24]:
from sklearn.metrics import roc_curve
import pickle
from google.colab import files
import time
from sklearn.externals import joblib

# Treina os métodos utilizados
scores = []
for method, name in zip(methods, titles):
    print("Treinando: ",name)
    # method.fit(X_train, y_train)
    method.fit(X, y)
    scores.append(method.score(X_test, y_test))
    print("Classification accuracy {} = {}"
          .format(name, method.score(X_test, y_test)))
    result = pd.DataFrame({'predict': method.predict( X_test )})
    result['y_test'] = y_test.reset_index(drop=True)
    result.to_excel(name + ".xlsx", sheet_name='Sheet_name_1')
    pickle.dump(method, open(name + ".pkl","wb"))
    joblib.dump(method, name + ".job")
    if name == 'DecisionTree':
      df = pd.DataFrame(dt.feature_importances_, index = X.columns) # apenas para decision tree
      df.to_excel("Feature Importance " + name + ".xlsx", sheet_name='Sheet_name_1') # exporta os campos importantes
      exporta_graf (name, method)
      # method.roc_curve(X_test, y_test, size_x=8, size_y=8)
    print("\n")

Treinando:  LogisticRegresson
Classification accuracy LogisticRegresson = 0.9305279329902874




### Python sklearn.decomposition.TruncatedSVD() Examples

In [0]:
# def reduce_dimensionality(self, X, n_features):
#         """
#         Apply PCA or SVD to reduce dimension to n_features.
#         :param X:
#         :param n_features:
#         :return:
#         """
#         # Initialize reduction method: PCA or SVD
#         if self.is_pca == 'PCA':
#            reducer = PCA(n_components=n_features)

#         #reducer = PCA(n_components=n_features)
#         if self.is_pca == 'SVD':
#             reducer = TruncatedSVD(n_components=n_features)
        
#         # Fit and transform data to n_features-dimensional space
#         reducer.fit(X)
#         self.X = reducer.transform(X)
#         logging.debug("Reduced number of features to {0}".format(n_features))
#         logging.debug("Percentage explained: %s\n" % reducer.explained_variance_ratio_.sum())
#         return X 

### Diversos

In [26]:
'''
minha_arvore = rf
minha_arvore.fit(X_train,y_train)

result = pd.DataFrame({ 'predict': minha_arvore.predict( X_test)})
result['y_test'] = y_test.reset_index(drop=True)
result.to_excel("Predicao_Resultado.xlsx", sheet_name='Sheet_name_1')

qtde_acertos = (result['predict'] == result['y_test']).sum()
qtde_obs = result.shape[0]

print('\n')
print('Acerto de {} carteiras em {}' .format(qtde_acertos,qtde_obs))
print('Acurácia: {} / {} = {}' .format(qtde_acertos, qtde_obs, qtde_acertos/qtde_obs) )

print("Classification accuracy = ", minha_arvore.score(X_test, y_test))

acc_sklearn = metrics.accuracy_score(result['y_test'] , result['predict'])
print('Accuracy score: ',acc_sklearn)

#precision = metrics.precision_score(result['y_test'],result['predict'])
#recall = metrics.recall_score(result['y_test'],result['predict'])
'''

'\nminha_arvore = rf\nminha_arvore.fit(X_train,y_train)\n\nresult = pd.DataFrame({ \'predict\': minha_arvore.predict( X_test)})\nresult[\'y_test\'] = y_test.reset_index(drop=True)\nresult.to_excel("Predicao_Resultado.xlsx", sheet_name=\'Sheet_name_1\')\n\nqtde_acertos = (result[\'predict\'] == result[\'y_test\']).sum()\nqtde_obs = result.shape[0]\n\nprint(\'\n\')\nprint(\'Acerto de {} carteiras em {}\' .format(qtde_acertos,qtde_obs))\nprint(\'Acurácia: {} / {} = {}\' .format(qtde_acertos, qtde_obs, qtde_acertos/qtde_obs) )\n\nprint("Classification accuracy = ", minha_arvore.score(X_test, y_test))\n\nacc_sklearn = metrics.accuracy_score(result[\'y_test\'] , result[\'predict\'])\nprint(\'Accuracy score: \',acc_sklearn)\n\n#precision = metrics.precision_score(result[\'y_test\'],result[\'predict\'])\n#recall = metrics.recall_score(result[\'y_test\'],result[\'predict\'])\n'

In [27]:
predict = lr.predict(X_test)
matrix = confusion_matrix(y_test, predict)

print(matrix)

[[ 454    0    0 ...    0    0    0]
 [   0  178    0 ...    0    0    0]
 [   0    0    8 ...    0    0    0]
 ...
 [   0    0    0 ... 1971    1    0]
 [   0    0    0 ...    0  748    0]
 [   0    0    0 ...    0    0   15]]


In [0]:
# EXEMPLO USANDO VALIDAÇÃO CRUZADA

#clfb = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
clfb = lr
folds=10
result = model_selection.cross_val_score(clfb, X, y, cv=folds)
print("\nCross Validation Results %d folds:" % folds)
print("Mean Accuracy: %.2f" % result.mean())
print("Mean Std: %.2f" % result.std())

### Confusion matrix

In [0]:
'''
fig3, sub1 = plt.subplots(5, 2, figsize=(15, 15))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
for clf, ax, title in zip(methods, sub1.flatten(), titles):
    y_predicted = clf.predict(X_test)
    cm = confusion_matrix(y_test , y_predicted)
    #plot_confusion_matrix(ax, cm, title)
    df_cm = pd.DataFrame(cm, index = [i for i in "0123456789"],
                  columns = [i for i in "0123456789"])
    sns.heatmap(df_cm, annot=True, ax=ax)
    ax.set_title('Confusion Matrix --> ' + title)
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')

plt.show()
plt.tight_layout()
''' 

### Plotando borda de classificação

In [0]:
'''
import matplotlib.pyplot as plt
from sklearn.manifold.t_sne import TSNE

methods = [minha_arvore]
titles = ['Decision Tree']
X_Embedded_train=X_train
X_Embedded_train = TSNE(n_components=2).fit_transform(X_train)

# Plotting the Classifier Decision
def plot_classifier_decision(ax, clf, X, X_embedded, mode='line', **params):
 
# create meshgrid
    resolution = 200 # 100x100 background pixels
    y_predicted = minha_arvore.predict(X);
    X2d_xmin, X2d_xmax = np.min(X_embedded[:,0]), np.max(X_embedded[:,0])
    X2d_ymin, X2d_ymax = np.min(X_embedded[:,1]), np.max(X_embedded[:,1])
    xx, yy = np.meshgrid(np.linspace(X2d_xmin, X2d_xmax, resolution), np.linspace(X2d_ymin, X2d_ymax, resolution))

# approximate Voronoi tesselation on resolution x resolution grid using 1-NN
    background_model = KNeighborsClassifier(n_neighbors=1).fit(X_embedded, y_predicted) 
    voronoiBackground = background_model.predict(np.c_[xx.ravel(), yy.ravel()])
    voronoiBackground = voronoiBackground.reshape((resolution, resolution))

#plot
    ax.contourf(xx, yy, voronoiBackground, levels=10, cmap="RdBu_r")
    plot_dataset(X_embedded, y_predicted, ax, 'teste', **params)

    return ax

# Plotting the Decision Border of the Classifiers.
fig2, sub = plt.subplots(2, 1, figsize=(10, 10))
plt.subplots_adjust(wspace=0.4, hspace=0.4)

for clf, ax, title in zip(methods, sub.flatten(), titles):
    plot_classifier_decision(ax, clf, X_train, X_Embedded_train, mode='filled', alpha=0.4)
    ax.set_title(title, fontsize=15)
# Setting figure to show
# sphinx_gallery_thumbnail_number = 3

plt.show()
plt.tight_layout()  
'''

### Criando o Flask

#### Montando o Flask

In [0]:
# from flask import Flask, request, redirect, url_for, flash, jsonify
# import numpy as np
# import pickle as p
# import json


# app = Flask(__name__)


# @app.route('/api/', methods=['POST'])
# def makecalc():
#     data = request.get_json()
#     prediction = np.array2string(model.predict(data))

#     return jsonify(prediction)

# if __name__ == '__main__':
#     modelfile = 'DecisionTree Entropy.pkl'
#     model = p.load(open(modelfile, 'rb'))
#     app.run(debug=True, host='0.0.0.0')

#### Gera a request

In [0]:
# import requests
# import json

# url = 'http://0.0.0.0:5000/api/'

# data = [[14.34, 1.68, 2.7, 25.0, 98.0, 2.8, 1.31, 0.53, 2.7, 13.0, 0.57, 1.96, 660.0]]
# j_data = json.dumps(data)
# headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# r = requests.post(url, data=j_data, headers=headers)
# print(r, r.text)

In [0]:
# # load the model from disk
# loaded_model = pickle.load(open('DecisionTree Gini.job', 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [0]:
# #load joblib
# model = joblib.load('DecisionTree Gini.job')
# print('Atributos do Modelo:\n\nClasses:{}\nEstimator:{}'.format(model.classes_, model.estimator))

In [0]:
#creating and training a model
#serializing our model to a file called model.pkl
#import pickle
#pickle.dump(minha_arvore, open("model.pkl","wb"))